In [2]:
import pandas as pd
df = pd.read_csv("../output/select_result.csv")
df.shape

(540, 11)

In [6]:
df

,layout,floors,per_room_occ,responders,exit_combo,makespan_hms,room_clear_order,exit_combo_id,makespan_s,responder_orders,start_positions
0,BASELINE,1,5,1,L,00:10:16,R3_F1->R0_F1->R1_F1->R4_F1->R5_F1->R2_F1,0,615.609835,F1:R3_F1->R0_F1->R1_F1->R4_F1->R5_F1->R2_F1,F1:E_L
1,BASELINE,1,5,2,LR,00:07:38,R2_F1->R3_F1->R5_F1->R0_F1->R4_F1->R1_F1,0,458.224927,F1:R3_F1->R0_F1 | F2:R2_F1->R5_F1->R4_F1->R1_F1,F1:E_L;F2:E_R
2,BASELINE,1,5,3,RRL,00:05:02,R2_F1->R3_F1->R1_F1->R5_F1->R0_F1->R4_F1,1,302.431717,F1:R1_F1->R4_F1 | F2:R2_F1->R5_F1 | F3:R3_F1->...,F1:E_R;F2:E_R;F3:E_L
3,BASELINE,1,5,4,LLRR,00:05:02,R2_F1->R0_F1->R3_F1->R1_F1->R5_F1->R4_F1,0,302.431717,F1:R0_F1 | F2:R3_F1 | F3:R1_F1->R4_F1 | F4:R2_...,F1:E_L;F2:E_L;F3:E_R;F4:E_R
4,BASELINE,1,5,5,RRRLL,00:04:50,R5_F1->R2_F1->R0_F1->R3_F1->R4_F1->R1_F1,1,290.426460,F1:R5_F1->R1_F1 | F2:R2_F1 | F3:R4_F1 | F4:R0_...,F1:E_R;F2:E_R;F3:E_R;F4:E_L;F5:E_L
...,...,...,...,...,...,...,...,...,...,...,...
535,L,18,5,6,LLLRRR,04:05:41,LV1_F1->LV2_F1->LH4_F3->LH4_F2->LV3_F1->LH3_F2...,0,14741.466375,F1:LV1_F1->LH2_F1->LV4_F1->LH1_F2->LV3_F2->LV3...,F1:E_L;F2:E_L;F3:E_L;F4:E_R;F5:E_R;F6:E_R
536,L,18,5,7,RRRRLLL,03:40:33,LV1_F1->LH4_F3->LV2_F1->LV3_F1->LH3_F3->LH4_F1...,1,13233.474638,F1:LH4_F3->LH4_F2->LH3_F1->LH4_F5->LV1_F5->LV5...,F1:E_R;F2:E_R;F3:E_R;F4:E_R;F5:E_L;F6:E_L;F7:E_L
537,L,18,5,8,LLLLRRRR,03:40:33,LV1_F1->LV2_F1->LH4_F3->LV3_F1->LV1_F2->LH3_F3...,0,13233.474638,F1:LV1_F1->LV5_F1->LV2_F2->LH1_F2->LV4_F3 | F2...,F1:E_L;F2:E_L;F3:E_L;F4:E_L;F5:E_R;F6:E_R;F7:E...
538,L,18,5,9,RRRRRLLLL,03:27:36,LV1_F1->LH4_F3->LV2_F1->LH4_F2->LV3_F1->LV1_F2...,1,12456.109122,F1:LH4_F4->LV4_F4->LH3_F1->LV5_F3->LH1_F6->LV1...,F1:E_R;F2:E_R;F3:E_R;F4:E_R;F5:E_R;F6:E_L;F7:E...


In [ ]:
import matplotlib.pyplot as plt

# 按 layout / floors / responders 聚合，取每种配置的最小完成时间（秒）
summary = (
    df.groupby(["layout", "floors", "responders"], as_index=False)["makespan_s"]
      .min()
)
summary["makespan_min_min"] = summary["makespan_s"] / 60  # 转为分钟，便于阅读

display(summary.head())

# 1）对每种 layout，查看在不同楼层下，响应人数对完成时间的影响
for layout in summary["layout"].unique():
    layout_df = summary[summary["layout"] == layout]
    floors_sorted = sorted(layout_df["floors"].unique())

    # 为满足“单个画布最多 3x3 子图”，按每 9 层一组拆分
    for i in range(0, len(floors_sorted), 9):
        group = floors_sorted[i:i + 9]
        n = len(group)
        ncols = min(3, n)
        nrows = (n + ncols - 1) // ncols
        fig, axes = plt.subplots(
            nrows,
            ncols,
            figsize=(4 * ncols, 3 * nrows),
            sharey=True,
        )

        if nrows == 1 and ncols == 1:
            axes = [axes]
        else:
            axes = axes.flatten()

        for ax, f in zip(axes, group):
            sub = layout_df[layout_df["floors"] == f].sort_values("responders")
            ax.plot(sub["responders"], sub["makespan_min_min"], marker="o")
            ax.set_title(f"{layout}, floors={f}")
            ax.set_xlabel("Number of responders")
            ax.grid(True, alpha=0.3)

        # 隐藏多余子图
        for ax in axes[len(group):]:
            ax.set_visible(False)

        axes[0].set_ylabel("Makespan (minutes)")
        fig.suptitle(f"Effect of responders on makespan - layout={layout}")
        plt.tight_layout()
        plt.show()

# 2）对每个楼层数，对比不同 layout 在响应人数变化下的完成时间
floors_sorted = sorted(summary["floors"].unique())
for f in floors_sorted:
    sub_floor = summary[summary["floors"] == f]
    if sub_floor.empty:
        continue
    fig, ax = plt.subplots(figsize=(5, 3))
    for layout in sub_floor["layout"].unique():
        sub = sub_floor[sub_floor["layout"] == layout].sort_values("responders")
        ax.plot(
            sub["responders"],
            sub["makespan_min_min"],
            marker="o",
            label=f"layout={layout}"
        )
    ax.set_title(f"floors={f}: layout vs responders")
    ax.set_xlabel("Number of responders")
    ax.set_ylabel("Makespan (minutes)")
    ax.grid(True, alpha=0.3)
    ax.legend()
    plt.tight_layout()
    plt.show()

### 可视化结论与建议（基于 `select_result.csv`）\n\n结合上面的折线图，可以得到以下观察：\n\n1. **增加响应人数显著降低整体疏散时间，但收益递减**：\n   - 对于 `BASELINE` 布局，1 层楼时从 1 人增加到 3 人，完成时间从约 10.3 分钟降低到约 5.0 分钟，下降接近一半；再继续从 3 人增加到 7 人，时间降到约 3.6 分钟，此后再增加（7→10 人）基本不再明显改善。\n   - `T` 和 `L` 布局在各楼层上也呈现类似特征：**1→3 人的效果最大，3 人之后每多 1–2 人的边际收益明显减少**，并且常常出现两个人数一组的“平台期”（例：3 和 4 人结果相同，5 和 6 人结果相同）。\n\n2. **楼层数增加，疏散时间近似线性增长**：\n   - 以 `BASELINE` 布局为例，在 10 名响应者情况下，1 层楼约 3.6 分钟，2 层楼约 6.4 分钟，6 层楼约 24.4 分钟，18 层楼约 165 分钟。楼层越高，总疏散时间基本随楼层数近似线性上升。\n   - `T` 和 `L` 布局中，高楼层（例如 18 层）下的疏散时间也远高于低楼层，说明**建筑高度本身是决定疏散时间的关键因素之一**。\n\n3. **不同布局的可疏散性差异明显**：\n   - 在相同楼层、相同响应人数下，`BASELINE` 布局的完成时间普遍低于 `T` 和 `L` 布局，例如：\n     - 1 层、10 名响应者：`BASELINE` ≈ 3.6 分钟，`L` ≈ 4.4 分钟，`T` ≈ 5.1 分钟。\n     - 18 层、10 名响应者：`BASELINE` ≈ 165 分钟，`L` ≈ 207 分钟，`T` ≈ 217 分钟。\n   - 说明在相同条件下，**走廊更简单（接近“直线”）的 `BASELINE` 布局更有利于快速疏散**，而 `T`、`L` 等复杂几何会增加路径长度和交汇干扰，从而延长疏散时间。\n\n4. **响应人数的“合理区间”**：\n   - 对多数布局-楼层组合，3 名响应者就能获得明显的时间改善，5–7 名响应者通常可以接近该配置的“最优或次优时间”。\n   - 超过 7 名响应者（例如 8–10 人），在很多楼层上时间改善非常有限，甚至完全相同，说明此时**受限因素已经从“人手不足”转变为“通道容量/路径结构”**。\n\n综合上述可视化结果，可以提出一些**建议**：\n\n- **响应人力配置建议**：在当前模拟的疏散策略下，对于小高层和中高层建筑，推荐优先保障每个布局至少配备 **3–5 名响应者**；如果资源允许，可扩展到 7 名左右作为上限，超过 7 名的边际收益相对较小。\n- **建筑设计建议**：尽量减少类似 `T`、`L` 形状的复杂走廊结构，或在转角、交汇处增加出口/楼梯间，以降低复杂布局带来的额外疏散时间。\n- **高层建筑策略**：对于 10 层以上建筑，即使增加响应人数，疏散时间仍然远高于低层建筑，应在规划阶段综合考虑 **增加楼梯间密度、分区疏散** 等方式，从源头缩短疏散距离。\n- **后续分析方向**：可以在此基础上进一步细化到具体的 `exit_combo`、`room_clear_order` 等路径特征，分析哪些出口组合和清点顺序最接近理论最优，并针对性优化疏散预案。